### OpenHoliday Connection

#### Global config

In [18]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import requests
import psycopg2
import datetime
import json
import time
import string
import uuid
import os
import sys

In [2]:
# Set the working directory explicitly
os.chdir('C:/Users/A/Documents/HS23_DWL/Git/DWLadies')

# Verify the current working directory
print("Current Working Directory:", os.getcwd())
# Print the content of the working directory
print("Directory Contents:", os.listdir())

sys.path.insert(0, os.getcwd())
print(sys.path)

# Try to import the Config module
try:
    from myconfig.config import config
    
    config_data = config.get_config()
    
    hostname = config_data["hostname"]
    port = config_data["port"]
    database = config_data["database"]
    username = config_data["username"]
    password = config_data["password"]
    PERSONAL_ACCESS_TOKEN = config_data["PERSONAL_ACCESS_TOKEN"]
    
    print("Config module imported successfully.")
    
except ImportError as e:
    print("Error importing Config module:", e)

Current Working Directory: C:\Users\A\Documents\HS23_DWL\Git\DWLadies
Directory Contents: ['.git', '.gitignore', '.ipynb_checkpoints', 'ConnectDBtoPostgres.txt', 'Holiday_API_Connection.ipynb', 'Holiday_Lambda_Function.ipynb', 'install_lambda_layer_python_3.9.txt', 'myconfig', 'Package', 'Package1', 'README.md', 'Weather_API_Connection.ipynb', 'Weather_Lambda_Function.ipynb', '__init__.ipynb']
['C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\anaconda3\\python39.zip', 'C:\\Users\\A\\anaconda3\\DLLs', 'C:\\Users\\A\\anaconda3\\lib', 'C:\\Users\\A\\anaconda3', '', 'C:\\Users\\A\\anaconda3\\lib\\site-packages', 'C:\\Users\\A\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\A\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\A\\anaconda3\\lib\\site-packages\\Pythonwin']
Error importing Config module: No module named 'myconfig.config'


#### API Documentation and Authentication

In [14]:
# All the entries we prefer to have in English
language =["EN"]
# Given neighbouring countries of Basel -> we are interested only Switzerland, Germany and France
countries =["CH","DE","FR"]

# basic API
API_URL = "https://openholidaysapi.org/"

# variables to customize API
typ = ["Languages","Countries","Subdivisions","PublicHolidays?","SchoolHolidays?"]
country_code = "?countryIsoCode=" + countries[0]
language_code = "&languageIsoCode=" + language[0]

# Dates current year & future (for the lambda function)
#date_from =  "&validFrom=" + str(datetime.date.today().year) +"-01-01"
#date_to =  "&validTo=" + str(datetime.date.today().year) +"-12-31"

# All dates from 2020 till 2030 (for initial download) 
years = list(range(2020, 2031))
for y in years:
    date_from = "&validFrom=" + str(y) + "-12-01"
    date_to = "&validTo=" + str(y+1) + "-11-30"
    print(date_from, "and", date_to)


&validFrom=2020-12-01 and &validTo=2021-11-30
&validFrom=2021-12-01 and &validTo=2022-11-30
&validFrom=2022-12-01 and &validTo=2023-11-30
&validFrom=2023-12-01 and &validTo=2024-11-30
&validFrom=2024-12-01 and &validTo=2025-11-30
&validFrom=2025-12-01 and &validTo=2026-11-30
&validFrom=2026-12-01 and &validTo=2027-11-30
&validFrom=2027-12-01 and &validTo=2028-11-30
&validFrom=2028-12-01 and &validTo=2029-11-30
&validFrom=2029-12-01 and &validTo=2030-11-30
&validFrom=2030-12-01 and &validTo=2031-11-30


In [3]:
# API Links to our holiday database

# API available languages
LANGUAGE_API = API_URL + typ[0]
print(LANGUAGE_API)

# API available countries
COUNTRY_API = API_URL + typ[1]
print(COUNTRY_API)

# API country subdivisions
DIVISION_API = API_URL + typ[2] + country_code
print(DIVISION_API)

for i in range(3,5):
    # API school and public holidays
    HOLIDAY_API = API_URL + str(typ[i]) + country_code + language_code + date_from + date_to
    print(HOLIDAY_API)


https://openholidaysapi.org/Languages
https://openholidaysapi.org/Countries
https://openholidaysapi.org/Subdivisions?countryIsoCode=CH
https://openholidaysapi.org/PublicHolidays??countryIsoCode=CH&languageIsoCode=EN&validFrom=2030-01-01&validTo=2030-12-31
https://openholidaysapi.org/SchoolHolidays??countryIsoCode=CH&languageIsoCode=EN&validFrom=2030-01-01&validTo=2030-12-31


In [4]:
# Make an API request

response = requests.get(LANGUAGE_API)
data = response.json()  # Assumes the API response is in JSON format
print(data)


[{'isoCode': 'BG', 'name': [{'language': 'EN', 'text': 'Bulgarian'}, {'language': 'BG', 'text': 'Български'}, {'language': 'DE', 'text': 'Bulgarisch'}]}, {'isoCode': 'CS', 'name': [{'language': 'EN', 'text': 'Czech'}, {'language': 'CS', 'text': 'Česky'}, {'language': 'DE', 'text': 'Tschechisch'}]}, {'isoCode': 'DE', 'name': [{'language': 'EN', 'text': 'German'}, {'language': 'DE', 'text': 'Deutsch'}]}, {'isoCode': 'EN', 'name': [{'language': 'EN', 'text': 'English'}, {'language': 'DE', 'text': 'Englisch'}]}, {'isoCode': 'ET', 'name': [{'language': 'EN', 'text': 'Estonian'}, {'language': 'ET', 'text': 'Eesti'}, {'language': 'DE', 'text': 'Estnisch'}]}, {'isoCode': 'FR', 'name': [{'language': 'EN', 'text': 'French'}, {'language': 'FR', 'text': 'français'}, {'language': 'DE', 'text': 'Französisch'}]}, {'isoCode': 'GA', 'name': [{'language': 'EN', 'text': 'Irish'}, {'language': 'GA', 'text': 'Gaeilge'}, {'language': 'DE', 'text': 'Irisch'}]}, {'isoCode': 'HR', 'name': [{'language': 'EN', '

In [5]:
# Connection parameters
hostname = "database-1-instance-1.cvqnug2g93ia.us-east-1.rds.amazonaws.com"
port = "5432"
database = "db_holiday"
username = "postgresadmin"
password = "DaSci_1122"


In [6]:
# connection to PostgresDB

try: 
    conn = psycopg2.connect(
        host=hostname,
        port=port,
        dbname=database,
        user=username,
        password=password)
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
# Auto commit is very important
conn.set_session(autocommit=True)


In [20]:
 # Define tables in database:
    
#delete_table =  """DROP TABLE holidays;"""
empty_table = """TRUNCATE TABLE holidays;"""

create_table_language = """
CREATE TABLE IF NOT EXISTS languages (
    isoCode_lan text PRIMARY KEY,
    name_lan TEXT
);
"""
create_table_countries = """
CREATE TABLE IF NOT EXISTS countries (
    isoCode_ctr text PRIMARY KEY,
    name_ctr TEXT
);
"""

create_table_subdivisions = """
CREATE TABLE IF NOT EXISTS subdivisions (
    code_div text PRIMARY KEY,
    shortName_div TEXT,
    category_div TEXT,
    name_div TEXT,
    officialLanguages TEXT[] NULL
);
"""

create_table_holidays = """
CREATE TABLE IF NOT EXISTS holidays (
    id_holiday uuid PRIMARY KEY,
    isoCode_holiday TEXT,
    start_date DATE,
    end_date DATE,
    type_holiday TEXT,
    name_holiday TEXT,
    nationwide BOOLEAN,
    subdivisions_holiday TEXT[] NULL
);
"""


In [21]:
# Connect cursor
cur = conn.cursor()

#cur.execute(delete_table)
cur.execute(empty_table)


In [9]:
# add UUID extension

# Connect cursor
cur = conn.cursor()

create_uuid = """CREATE EXTENSION IF NOT EXISTS "uuid-ossp";"""
cur.execute(create_uuid)


In [22]:
# Create tables in postgres database

# Connect cursor
cur = conn.cursor()

# Execute query in database using `cur`
cur.execute(create_table_language)
cur.execute(create_table_countries)
cur.execute(create_table_subdivisions)
cur.execute(create_table_holidays)

# Commit transaction (if not autocommit)
#conn.commit()


In [45]:
# Insert data into Postgres database: languages table

response = requests.get(LANGUAGE_API)
data = response.json()  # Assumes the API response is in JSON format

# Connect cursor
cur = conn.cursor()

# Inserting values from json file
try: 
    # Insert each data object into the table
    for item in data:
        isoCode_lan = item["isoCode"]
        # we substract only the name of the language in English from the nested structure
        name_lan = json.dumps(item["name"][0]["text"])
        query = f"INSERT INTO languages (isoCode_lan, name_lan) VALUES (%s, %s)"
        cur.execute(query, (isoCode_lan, name_lan))
        print("successfully finished")
        
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

# Commit transaction (if not autocommit)
#conn.commit()


In [46]:
# Insert data into Postgres database: countries table

response = requests.get(COUNTRY_API)
data = response.json()  # Assumes the API response is in JSON format

# Connect cursor
cur = conn.cursor()

# Inserting values from json file
try: 
    # Insert each data object into the table
    for item in data:
        isoCode_ctr = item["isoCode"]
        # we substract only the name of the language in English from the nested structure
        name_ctr = json.dumps(item["name"][0]["text"])
        query = f"INSERT INTO countries (isoCode_ctr, name_ctr) VALUES (%s, %s)"
        cur.execute(query, (isoCode_ctr, name_ctr))
        print("successfully finished")
        
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


In [114]:
# Create function: data insert subdivisions table

def insert_subdivisions(data):
            
    for item in data:
        code_div = item["code"]
        shortName_div = item["shortName"]
        category_div = [entry["text"] for entry in item["category"] if entry ["language"] == "EN"]
        name_div = [entry["text"] for entry in item["name"] if entry ["language"] == "EN"]
        officialLanguages = item.get("officialLanguages", [])
        
        if code_div.startswith("FR") and "children" in item:
            children_data = item["children"]
            for child_item in children_data:
                code_div = child_item["code"]
                shortName_div = child_item["shortName"]
                # Extract category_div with language "EN"
                category_div = [entry["text"] for entry in child_item["category"] if entry["language"] == "EN"]
                # Extract name_div with language "FR"
                name_div = [entry["text"] for entry in child_item["name"] if entry["language"] == "FR"]
                
                cur.execute(
                    "INSERT INTO subdivisions (code_div, shortName_div, category_div, name_div, officialLanguages) VALUES (%s, %s, %s, %s, %s)",
                    (code_div, shortName_div, category_div, name_div, officialLanguages)
                )
        else:
            cur.execute(
                "INSERT INTO subdivisions (code_div, shortName_div, category_div, name_div, officialLanguages) VALUES (%s, %s, %s, %s, %s)",
                (code_div, shortName_div, category_div, name_div, officialLanguages)
            )
        
        

In [115]:
# Insert data into Postgres database: subdivisions table

for country in countries:
    country_code = "?countryIsoCode=" + country
    DIVISION_API = API_URL + typ[2] + country_code
    print(DIVISION_API)
    response = requests.get(DIVISION_API)
    data = response.json()
    
    # Connect cursor
    cur = conn.cursor()
    
    try:
        insert_subdivisions(data)
        print("successfully finished")
        
    except psycopg2.Error as e: 
        print("Error: Inserting Rows")
        print (e)


https://openholidaysapi.org/Subdivisions?countryIsoCode=CH
successfully finished
https://openholidaysapi.org/Subdivisions?countryIsoCode=DE
successfully finished
https://openholidaysapi.org/Subdivisions?countryIsoCode=FR
successfully finished


In [23]:
# Create function: data insert public holidays table

def insert_holidays(data):
    for item in data:
        id_holiday = item["id"]
        isoCode_holiday = country,
        start_date = item["startDate"]
        end_date = item["endDate"]
        type_holiday = item["type"]
        name_holiday = [entry["text"] for entry in item["name"] if entry ["language"] == "EN"]
        nationwide = item["nationwide"]
        subdivisions_holiday = item.get("code", [])
        if "subdivisions" in item:
            subdivisions_holiday = [entry["code"] for entry in item["subdivisions"]]
        else:
            subdivisions_holiday = []
        
        cur.execute(
                "INSERT INTO holidays (id_holiday, isoCode_holiday, start_date, end_date, type_holiday, name_holiday, nationwide, subdivisions_holiday) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                (id_holiday, isoCode_holiday, start_date, end_date, type_holiday, name_holiday, nationwide, subdivisions_holiday)
            )
        
        

In [25]:
# Insert data into Postgres database: public_holidays

for country in countries:
    country_code = "countryIsoCode=" + country
    for y in years:
        date_from = "&validFrom=" + str(y) + "-12-01"
        date_to = "&validTo=" + str(y+1) + "-11-30"
        for i in range(3,5):
            HOLIDAY_API = API_URL + typ[i] + country_code + language_code + date_from + date_to
            print(HOLIDAY_API)
            response = requests.get(HOLIDAY_API)
            data = response.json()
        
            # Check if the list is empty, if empty do not insert
            if not data:  
                print("The holidaylist for is empty. Continuing...")
            else:
                # Connect cursor
                cur = conn.cursor()

                try:
                    insert_holidays(data)
                    print("successfully finished")

                except psycopg2.Error as e: 
                    print("Error: Inserting Rows")
                    print (e)
                    
                  

https://openholidaysapi.org/PublicHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2020-12-01&validTo=2021-11-30
successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2020-12-01&validTo=2021-11-30
successfully finished
https://openholidaysapi.org/PublicHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2021-12-01&validTo=2022-11-30
successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2021-12-01&validTo=2022-11-30
successfully finished
https://openholidaysapi.org/PublicHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2022-12-01&validTo=2023-11-30
successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2022-12-01&validTo=2023-11-30
successfully finished
https://openholidaysapi.org/PublicHolidays?countryIsoCode=CH&languageIsoCode=EN&validFrom=2023-12-01&validTo=2024-11-30
successfully finished
https:

successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=FR&languageIsoCode=EN&validFrom=2026-12-01&validTo=2027-11-30
The holidaylist for is empty. Continuing...
https://openholidaysapi.org/PublicHolidays?countryIsoCode=FR&languageIsoCode=EN&validFrom=2027-12-01&validTo=2028-11-30
successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=FR&languageIsoCode=EN&validFrom=2027-12-01&validTo=2028-11-30
The holidaylist for is empty. Continuing...
https://openholidaysapi.org/PublicHolidays?countryIsoCode=FR&languageIsoCode=EN&validFrom=2028-12-01&validTo=2029-11-30
successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=FR&languageIsoCode=EN&validFrom=2028-12-01&validTo=2029-11-30
The holidaylist for is empty. Continuing...
https://openholidaysapi.org/PublicHolidays?countryIsoCode=FR&languageIsoCode=EN&validFrom=2029-12-01&validTo=2030-11-30
successfully finished
https://openholidaysapi.org/SchoolHolidays?countryIsoCode=FR

In [26]:
# Close the cursor and connection when done
cur.close()
conn.close()